In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
import math

In [3]:
df = pd.read_csv('../../proyecto_final/data/clean_data/apple&spotify_nofeatures.csv')

In [4]:
df.head()

,date,artistName,trackName,msPlayed,sgPlayed,minPlayed,cal_min,count_min,workout
0,2021-11-12 15:22:00,Myke Towers,Experimento,106394,106.394,1.773233,0.316418,100.0,No Workout
1,2021-11-13 08:53:00,Tame Impala,The Less I Know The Better,3712,3.712,0.061867,0.210588,67.0,No Workout
2,2021-11-13 08:56:00,Tame Impala,The Less I Know The Better,213141,213.141,3.552350,0.210588,67.0,No Workout
3,2021-11-13 09:33:00,Goth Babe,Canary Islands,3904,3.904,0.065067,0.316364,67.0,No Workout
4,2021-11-13 09:36:00,Goth Babe,Canary Islands,201042,201.042,3.350700,0.316364,67.0,No Workout


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15029 entries, 0 to 15028
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        15029 non-null  object 
 1   artistName  15029 non-null  object 
 2   trackName   15029 non-null  object 
 3   msPlayed    15029 non-null  int64  
 4   sgPlayed    15029 non-null  float64
 5   minPlayed   15029 non-null  float64
 6   cal_min     15029 non-null  float64
 7   count_min   15029 non-null  float64
 8   workout     15029 non-null  object 
dtypes: float64(4), int64(1), object(4)
memory usage: 1.0+ MB


In [6]:
df.shape

(15029, 9)

In [10]:
df = df.astype({'date':'datetime64[ns]'})

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15029 entries, 0 to 15028
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        15029 non-null  datetime64[ns]
 1   artistName  15029 non-null  object        
 2   trackName   15029 non-null  object        
 3   msPlayed    15029 non-null  int64         
 4   sgPlayed    15029 non-null  float64       
 5   minPlayed   15029 non-null  float64       
 6   cal_min     15029 non-null  float64       
 7   count_min   15029 non-null  float64       
 8   workout     15029 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 1.0+ MB


In [13]:
features = pd.read_csv('../../proyecto_final/data/clean_data/spotify_features.csv')

In [14]:
features.head()

,todo,trackName,artistName,artistLink,albumLink,albumType,albumReleaseDate,trackDuration,trackExplicit,trackPopularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres
0,"{'album': {'album_type': 'single', 'artists': ...",Experimento,Myke Towers,https://api.spotify.com/v1/artists/7iK8PXO48We...,https://open.spotify.com/album/1KKwufGzWD4zIKq...,single,2021-09-23,222728,False,75,...,6,-3.589,0,0.0393,0.08280,0.00000,0.1560,0.6820,110.042,"['reggaeton', 'trap latino', 'urbano latino']"
1,"{'album': {'album_type': 'single', 'artists': ...",Canary Islands,Goth Babe,https://api.spotify.com/v1/artists/7o96HO2zruj...,https://open.spotify.com/album/3T0gMffhvjVW4DQ...,single,2021-01-28,204521,False,62,...,1,-7.699,1,0.0523,0.02800,0.00329,0.1560,0.7280,115.058,"['indie rock', 'modern rock']"
2,"{'album': {'album_type': 'single', 'artists': ...",Some,Steve Lacy,https://api.spotify.com/v1/artists/57vWImR43h4...,https://open.spotify.com/album/4POiurFeB5bDsF9...,single,2016-10-31,118393,True,79,...,7,-6.417,1,0.0300,0.00548,0.25400,0.0995,0.6340,104.010,['afrofuturism']
3,"{'album': {'album_type': 'album', 'artists': [...",Feels Like We Only Go Backwards,Tame Impala,https://api.spotify.com/v1/artists/5INjqkS1o8h...,https://open.spotify.com/album/3C2MFZ2iHotUQOS...,album,2012-01-01,192960,False,75,...,2,-2.352,0,0.0574,0.02770,0.63500,0.1100,0.4910,149.508,"['australian psych', 'neo-psychedelic']"
4,"{'album': {'album_type': 'album', 'artists': [...",Selfless,The Strokes,https://api.spotify.com/v1/artists/0epOFNiUfyO...,https://open.spotify.com/album/2xkZV2Hl1Omi8rk...,album,2020-04-10,222093,False,70,...,4,-4.297,1,0.0333,0.19100,0.00175,0.0889,0.0888,121.990,"['alternative rock', 'garage rock', 'modern ro..."


In [15]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   todo              2176 non-null   object 
 1   trackName         2176 non-null   object 
 2   artistName        2176 non-null   object 
 3   artistLink        2176 non-null   object 
 4   albumLink         2176 non-null   object 
 5   albumType         2176 non-null   object 
 6   albumReleaseDate  2176 non-null   object 
 7   trackDuration     2176 non-null   int64  
 8   trackExplicit     2176 non-null   bool   
 9   trackPopularity   2176 non-null   int64  
 10  local             2176 non-null   bool   
 11  danceability      2176 non-null   float64
 12  energy            2176 non-null   float64
 13  key               2176 non-null   int64  
 14  loudness          2176 non-null   float64
 15  mode              2176 non-null   int64  
 16  speechiness       2176 non-null   float64


In [18]:
features.shape

(2176, 24)

In [16]:
features['id'] = features['trackName'] + '-' + features['artistName']

In [17]:
df['id'] = df['trackName'] + '-' + df['artistName']

In [23]:
features.drop_duplicates(inplace=True)

In [24]:
features.shape

(2157, 24)

In [27]:
pd.concat([df, features], axis=1, on='id', how='left')

AttributeError: 'DataFrame' object has no attribute 'concat'

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15029 entries, 0 to 15028
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        15029 non-null  datetime64[ns]
 1   artistName  15029 non-null  object        
 2   trackName   15029 non-null  object        
 3   msPlayed    15029 non-null  int64         
 4   sgPlayed    15029 non-null  float64       
 5   minPlayed   15029 non-null  float64       
 6   cal_min     15029 non-null  float64       
 7   count_min   15029 non-null  float64       
 8   workout     15029 non-null  object        
 9   id          15029 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 1.1+ MB


In [30]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2157 entries, 0 to 2174
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   todo              2157 non-null   object 
 1   trackName         2157 non-null   object 
 2   artistName        2157 non-null   object 
 3   artistLink        2157 non-null   object 
 4   albumLink         2157 non-null   object 
 5   albumType         2157 non-null   object 
 6   albumReleaseDate  2157 non-null   object 
 7   trackDuration     2157 non-null   int64  
 8   trackExplicit     2157 non-null   bool   
 9   trackPopularity   2157 non-null   int64  
 10  local             2157 non-null   bool   
 11  danceability      2157 non-null   float64
 12  energy            2157 non-null   float64
 13  key               2157 non-null   int64  
 14  loudness          2157 non-null   float64
 15  mode              2157 non-null   int64  
 16  speechiness       2157 non-null   float64


In [38]:
df = df.astype({'id':'str'})

In [39]:
features = features.astype({'id':'str'})

In [43]:
total = df.merge(features, on='id', how='left')

In [45]:
total.duplicated().sum()

1

In [46]:
col_nan = total.isna().sum()
col_nan[col_nan>0]

todo                9562
trackName_y         9562
artistName_y        9562
artistLink          9562
albumLink           9562
albumType           9562
albumReleaseDate    9562
trackDuration       9562
trackExplicit       9562
trackPopularity     9562
local               9562
danceability        9562
energy              9562
key                 9562
loudness            9562
mode                9562
speechiness         9562
acousticness        9562
instrumentalness    9562
liveness            9562
valence             9562
tempo               9562
genres              9562
dtype: int64

In [64]:
total.columns

Index(['date', 'artistName_x', 'trackName_x', 'msPlayed', 'sgPlayed',
       'minPlayed', 'cal_min', 'count_min', 'workout', 'id', 'todo',
       'trackName_y', 'artistName_y', 'artistLink', 'albumLink', 'albumType',
       'albumReleaseDate', 'trackDuration', 'trackExplicit', 'trackPopularity',
       'local', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'genres'],
      dtype='object')

In [69]:
total.shape

(15184, 33)

In [76]:
total.drop({'trackName_y'}, axis=1, inplace=True)

In [77]:
total.columns

Index(['date', 'artistName_x', 'trackName_x', 'msPlayed', 'sgPlayed',
       'minPlayed', 'cal_min', 'count_min', 'workout', 'id', 'todo',
       'artistLink', 'albumLink', 'albumType', 'albumReleaseDate',
       'trackDuration', 'trackExplicit', 'trackPopularity', 'local',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'genres'],
      dtype='object')

In [88]:
duplicate = total[total.duplicated(subset='date')]

In [91]:
duplicate.head()

,date,artistName_x,trackName_x,msPlayed,sgPlayed,minPlayed,cal_min,count_min,workout,id,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres
949,2021-11-30 12:01:00,Jon Lajoie,Everyday Normal Guy 2,195693,195.693,3.261550,0.017120,80.0,No Workout,Everyday Normal Guy 2-Jon Lajoie,...,3.0,-4.273,1.0,0.307,0.420,0.00000,0.3060,0.876,90.302,"['comedy rap', 'comic', 'parody']"
959,2021-11-30 12:25:00,Jon Lajoie,Everyday Normal Guy 2,195693,195.693,3.261550,0.017120,80.0,No Workout,Everyday Normal Guy 2-Jon Lajoie,...,3.0,-4.273,1.0,0.307,0.420,0.00000,0.3060,0.876,90.302,"['comedy rap', 'comic', 'parody']"
1071,2021-12-04 13:10:00,Jon Lajoie,Everyday Normal Guy 2,195693,195.693,3.261550,0.195694,102.0,No Workout,Everyday Normal Guy 2-Jon Lajoie,...,3.0,-4.273,1.0,0.307,0.420,0.00000,0.3060,0.876,90.302,"['comedy rap', 'comic', 'parody']"
1275,2021-12-10 14:42:00,YEИDRY,Nena - A COLORS SHOW,153251,153.251,2.554183,0.032174,93.0,No Workout,Nena - A COLORS SHOW-YEИDRY,...,10.0,-7.462,0.0,0.341,0.751,0.00122,0.0839,0.723,154.931,['dominican pop']
1575,2021-12-15 19:59:00,Valen Madanes,DELINCUENTE,165529,165.529,2.758817,0.090128,71.0,No Workout,DELINCUENTE-Valen Madanes,...,0.0,-4.812,1.0,0.067,0.266,0.00000,0.0494,0.534,89.989,[]


In [92]:
duplicate.shape

(156, 31)

## Revisar qué canciones falta link y scrapeo

In [97]:
nan = total[total.acousticness.isna()]

In [98]:
nan.shape

(9562, 31)

In [100]:
nan.tail()

,date,artistName_x,trackName_x,msPlayed,sgPlayed,minPlayed,cal_min,count_min,workout,id,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres
15179,2022-11-13 13:34:00,Ozuna,Somos Iguales,172414,172.414,2.873567,0.421935,79.0,No Workout,Somos Iguales-Ozuna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15180,2022-11-13 13:35:00,NATHY PELUSO,LA DESPEDIDA - Live - Spotify Singles,88826,88.826,1.480433,0.421935,79.0,No Workout,LA DESPEDIDA - Live - Spotify Singles-NATHY PE...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15181,2022-11-13 13:38:00,Jonas Blue,Mama,184133,184.133,3.068883,0.421935,79.0,No Workout,Mama-Jonas Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15182,2022-11-13 13:40:00,Tokischa,Sistema De Patio,134222,134.222,2.237033,0.421935,79.0,No Workout,Sistema De Patio-Tokischa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15183,2022-11-13 13:43:00,David Guetta,Flames,177864,177.864,2.964400,0.421935,79.0,No Workout,Flames-David Guetta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9562 entries, 1 to 15183
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              9562 non-null   datetime64[ns]
 1   artistName_x      9562 non-null   object        
 2   trackName_x       9562 non-null   object        
 3   msPlayed          9562 non-null   int64         
 4   sgPlayed          9562 non-null   float64       
 5   minPlayed         9562 non-null   float64       
 6   cal_min           9562 non-null   float64       
 7   count_min         9562 non-null   float64       
 8   workout           9562 non-null   object        
 9   id                9562 non-null   object        
 10  todo              0 non-null      object        
 11  artistLink        0 non-null      object        
 12  albumLink         0 non-null      object        
 13  albumType         0 non-null      object        
 14  albumReleaseDate  0 non

In [106]:
nan = nan.drop_duplicates(subset='trackName_x', ignore_index=True)

In [107]:
nan.head()

,date,artistName_x,trackName_x,msPlayed,sgPlayed,minPlayed,cal_min,count_min,workout,id,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres
0,2021-11-13 08:53:00,Tame Impala,The Less I Know The Better,3712,3.712,0.061867,0.210588,67.0,No Workout,The Less I Know The Better-Tame Impala,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-11-13 09:48:00,The Marías,Hush - Still Woozy Remix,182347,182.347,3.039117,0.316364,67.0,No Workout,Hush - Still Woozy Remix-The Marías,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-11-13 10:19:00,ROSALÍA,LA FAMA (with The Weeknd),367373,367.373,6.122883,0.316364,67.0,No Workout,LA FAMA (with The Weeknd)-ROSALÍA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-11-13 10:24:00,Natos y Waor,Sudores fríos,275878,275.878,4.597967,0.316364,67.0,No Workout,Sudores fríos-Natos y Waor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-11-13 10:28:00,C. Tangana,Párteme La Cara,167867,167.867,2.797783,0.316364,67.0,No Workout,Párteme La Cara-C. Tangana,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
nan['search'] = nan['trackName_x'] + ' ' + nan['artistName_x']

/var/folders/s4/79ykyqjn7f16bdl_zqjgsbzh0000gp/T/ipykernel_54359/2370342285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan['search'] = nan['trackName_x'] + ' ' + nan['artistName_x']


In [111]:
nan['search'] = nan.search.str.replace(' ','%20')

/var/folders/s4/79ykyqjn7f16bdl_zqjgsbzh0000gp/T/ipykernel_54359/1122414555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan['search'] = nan.search.str.replace(' ','%20')


In [112]:
nan.head()

,date,artistName_x,trackName_x,msPlayed,sgPlayed,minPlayed,cal_min,count_min,workout,id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres,search
0,2021-11-13 08:53:00,Tame Impala,The Less I Know The Better,3712,3.712,0.061867,0.210588,67.0,No Workout,The Less I Know The Better-Tame Impala,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The%20Less%20I%20Know%20The%20Better%20Tame%20...
1,2021-11-13 09:48:00,The Marías,Hush - Still Woozy Remix,182347,182.347,3.039117,0.316364,67.0,No Workout,Hush - Still Woozy Remix-The Marías,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hush%20-%20Still%20Woozy%20Remix%20The%20Marías
2,2021-11-13 10:19:00,ROSALÍA,LA FAMA (with The Weeknd),367373,367.373,6.122883,0.316364,67.0,No Workout,LA FAMA (with The Weeknd)-ROSALÍA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LA%20FAMA%20(with%20The%20Weeknd)%20ROSALÍA
3,2021-11-13 10:24:00,Natos y Waor,Sudores fríos,275878,275.878,4.597967,0.316364,67.0,No Workout,Sudores fríos-Natos y Waor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sudores%20fríos%20Natos%20y%20Waor
4,2021-11-13 10:28:00,C. Tangana,Párteme La Cara,167867,167.867,2.797783,0.316364,67.0,No Workout,Párteme La Cara-C. Tangana,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Párteme%20La%20Cara%20C.%20Tangana


In [114]:
nan['search'] = 'https://open.spotify.com/search/' + nan['search']

/var/folders/s4/79ykyqjn7f16bdl_zqjgsbzh0000gp/T/ipykernel_54359/2383855031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan['search'] = 'https://open.spotify.com/search/' + nan['search']


In [118]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
PATH=ChromeDriverManager().install()

In [120]:
CHROME_PATH = '/usr/bin/google-chrome'
CHROMEDRIVER_PATH = '/usr/bin/chromedriver'
WINDOW_SIZE = "1920,1080"
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.binary_location = CHROME_PATH

In [117]:
uri=[]

In [121]:
for i in tqdm(range(len(nan['search']))):
    
    #inicia el driver
    driver=webdriver.Chrome(PATH)
    driver.get(nan['search'][i])
    time.sleep(6)

    # acepta cookies
    try:
        aceptar=driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
        aceptar.click()

        time.sleep(2)
    
    except:
        print('error 0')      

    #Anuncio Premium
    try:
        anuncio=driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/footer/div[1]/button[2]')
        anuncio.click()
        time.sleep(1)

    except:
        print('no anuncio')

    # seleccionar las opciones de la canción
    try:
        opciones=driver.find_element(By.XPATH, '//*[@id="searchPage"]/div/div/section[2]/div[2]/div/div/div/div[2]/div[1]/div/div[2]/button[2]')
        opciones.click()
        time.sleep(1)
        
    except:
        print('error 2')

    #seleccionar compartir
    try:
        compartir=driver.find_element(By.XPATH, '//*[@id="context-menu"]/ul/li[7]/button/span[1]')
        compartir.click()
        time.sleep(2)
        
    except:
        print('error 3')

    #copiar el enlace de la canción
    try:
        copiarlink = driver.find_element(By.XPATH, '//*[@id="tippy-4"]/ul/li[1]/button/span')
        copiarlink.click()
    
    except:
        print('error 4')

    #pegar el enlace
    try: 
        uri.append(clipboard.paste())
        print(clipboard.paste())

    except:
        uri.append('0')
        print('error al pegar')
        
    #apagamos el driver
    driver.quit()

  0%|          | 0/3910 [00:00<?, ?it/s]

/var/folders/s4/79ykyqjn7f16bdl_zqjgsbzh0000gp/T/ipykernel_54359/3706541064.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)


error al pegar
no anuncio
error 4
error al pegar
no anuncio
error 4
error al pegar


KeyboardInterrupt: 

In [134]:
artist = pd.DataFrame(total.groupby(['artistName_x'])['cal_min'].sum())

In [135]:
artist.reset_index(drop=False, inplace=True)

In [142]:
artist.drop(artist['artistName_x']=='Jazz Relajante', axis=0, inplace=True)

KeyError: '[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, F

In [141]:
artist.sort_values(by=['cal_min'], ascending=False)[:10]

,artistName_x,cal_min
219,Bad Bunny,395.875314
1866,ROSALÍA,350.443970
1854,Quevedo,289.125295
274,Beyoncé,177.302282
2368,WILLOW,148.251626
1877,Rauw Alejandro,135.244929
190,Avril Lavigne,117.463126
1484,Manuel Turizo,111.631728
92,Alicia Keys,106.578428
1073,Jazz Relajante,105.202384


In [139]:
C_ar.artistName_x.unique()

array(['Bad Bunny', 'ROSALÍA', 'Quevedo', 'Beyoncé', 'WILLOW',
       'Rauw Alejandro', 'Avril Lavigne', 'Manuel Turizo', 'Alicia Keys',
       'Jazz Relajante'], dtype=object)

In [143]:
BB = total.loc[total['artistName_x']=='Bad Bunny']

In [149]:
BB.head()

,date,artistName_x,trackName_x,msPlayed,sgPlayed,minPlayed,cal_min,count_min,workout,id,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres
182,2021-11-14 21:55:00,Bad Bunny,120,151506,151.506,2.525100,0.128756,63.0,No Workout,120-Bad Bunny,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
772,2021-11-28 10:19:00,Bad Bunny,ANTES QUE SE ACABE,111362,111.362,1.856033,0.746346,71.0,No Workout,ANTES QUE SE ACABE-Bad Bunny,...,2.0,-6.098,1.0,0.0577,0.505,0.000047,0.3240,0.473,139.988,"['reggaeton', 'trap latino', 'urbano latino']"
975,2021-12-02 11:19:00,Bad Bunny,Amorfoda,153887,153.887,2.564783,0.259500,71.0,No Workout,Amorfoda-Bad Bunny,...,9.0,-7.925,0.0,0.0587,0.985,0.000126,0.0832,0.307,147.887,"['reggaeton', 'trap latino', 'urbano latino']"
1251,2021-12-10 12:05:00,Bad Bunny,LA NOCHE DE ANOCHE,203200,203.200,3.386667,0.032174,93.0,No Workout,LA NOCHE DE ANOCHE-Bad Bunny,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1688,2021-12-21 18:13:00,Bad Bunny,Soy Peor,650324,650.324,10.838733,0.171000,73.0,No Workout,Soy Peor-Bad Bunny,...,0.0,-3.221,0.0,0.0663,0.683,0.006380,0.1140,0.172,116.011,"['reggaeton', 'trap latino', 'urbano latino']"


In [155]:
BB.key.isna()

182       True
772      False
975      False
1251      True
1688     False
         ...  
14768     True
14770     True
14908     True
14911    False
15156     True
Name: key, Length: 496, dtype: bool

In [156]:
indice=[]

In [157]:
for i,e in enumerate(BB['key'].isna()):
    indice.append(i)

In [148]:
BB.trackName_x.unique()

array(['120', 'ANTES QUE SE ACABE', 'Amorfoda', 'LA NOCHE DE ANOCHE',
       'Soy Peor', 'Sensualidad', 'TE MUDASTE', 'DÁKITI', 'Yonaguni',
       'A Tu Merced', 'Vete', 'HACIENDO QUE ME AMAS', 'Si Veo a Tu Mamá',
       'Si Estuviésemos Juntos', 'La Difícil', 'Callaita', 'La Santa',
       'YO VISTO ASÍ', 'Ignorantes', 'Una Vez', 'La Zona', 'Diles',
       'TE DESEO LO MEJOR', 'Moscow Mule', 'Party',
       'Me Fui de Vacaciones', 'Un Verano Sin Ti', 'Agosto',
       'Después de la Playa', 'Me Porto Bonito', 'Tití Me Preguntó',
       'Un Ratito', 'Yo No Soy Celoso', 'Tarot', 'Neverita',
       'La Corriente', 'Efecto', 'Un Coco', 'Ojitos Lindos', 'Andrea',
       'El Apagón', 'BOOKER T', "Puesto Pa' Guerrial", 'Chambea', 'Pa Ti',
       'Que Malo', '200 Mph', 'La Romana', 'Otra Noche en Miami',
       'Otro Atardecer', 'Soliá', 'Enséñame a Bailar', 'Aguacero',
       'TRELLAS', 'SORRY PAPI', 'Dos Mil 16', 'Dime Si Te Acuerdas',
       'Está Cabrón Ser Yo', 'Estamos Bien'], dtype=obje